In [1]:
import os
import sys
spark_home = r"C:\spark\spark-4.0.0-bin-hadoop3"
sys.path.insert(0, spark_home + r"\python")
sys.path.insert(0, spark_home + r"\python\lib\py4j-0.10.9.9-src.zip")
os.environ["PYSPARK_PYTHON"] = "C:/Users/Raghava/AppData/Local/Programs/Python/Python310/python.exe"
os.environ["PYSPARK_DRIVER_PYTHON"] = "C:/Users/Raghava/AppData/Local/Programs/Python/Python310/python.exe"


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg,unix_timestamp, round as pyspark_round
from pyspark.sql.window import Window

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
activity_df = spark.read.csv("H:/leet_code_sql_50/sql_50_leetcode/sample_data/10_activity.csv", header=True, inferSchema=True)

In [5]:
# Split the activity table into start and end events
start_df = activity_df.filter(col("activity_type") == "start") \
    .selectExpr("machine_id", "process_id", "timestamp as start_time")

end_df = activity_df.filter(col("activity_type") == "end") \
    .selectExpr("machine_id", "process_id", "timestamp as end_time")

# Join start and end events on machine_id and process_id
joined_df = start_df.join(
    end_df,
    on=["machine_id", "process_id"]
).filter(col("start_time") < col("end_time"))

# Calculate duration and average per machine
result_df = joined_df.withColumn(
    "duration", (col("end_time").cast("long") - col("start_time").cast("long")).cast("double")
).groupBy("machine_id").agg(
    pyspark_round(avg("duration"), 3).alias("processing_time")
)

# Show result
result_df.show()

+----------+---------------+
|machine_id|processing_time|
+----------+---------------+
|         1|            1.0|
|         2|            1.5|
|         0|            1.0|
+----------+---------------+

